In [1]:
import numpy as np

# Non-Maximum Supression

In [108]:
# Faster Non-Maximum Supression

def non_maximum_suppression(boxes, thresh=0.3):
    """
    Reference: https://www.pyimagesearch.com/2015/02/16/faster-non-maximum-suppression-python/
    
    Parameters
    ----------
    boxes: NumPy array, size: [?, 5], where ? can be some int, and 5 specifies 
        x_min, y_min, x_max, y_max
    thresh: float
    """
    # Get the coordinates of the bounding boxes
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 2]
    y2 = boxes[:, 3]
    scores = boxes[:, 4]  # Confidence scores
    
    # Compute area of each bounding box
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    
    # Sort the bounding boxes by confidence score
    indices = scores.argsort()
    
    # Initialize a list for indices to keep
    keep = []
    
    while len(indices) > 0:
        
        # Grab the last index in the indices list and add the
        # index value to the keep list
        last = len(indices) - 1
        i = indices[last]
        keep.append(i)
        
        # Find the largest (x, y) coordinates for the start of the
        # bounding box and the smallest (x, y) coordinates for the
        # end of the bounding box
        xx1 = np.maximum(x1[i], x1[indices[:last]])
        yy1 = np.maximum(y1[i], y1[indices[:last]])
        xx2 = np.minimum(x2[i], x2[indices[:last]])
        yy2 = np.minimum(y2[i], y2[indices[:last]])
        
        # Compute the width and height of the bounding boxes
        w = np.maximum(0., xx2 - xx1 + 1)
        h = np.maximum(0., yy2 - yy1 + 1)
        
        # Compute the IoU
        inter_area = w * h
        iou = inter_area / (areas[i] + areas[indices[:last]] - inter_area)
        
        # Delete all the indices where 
        indices = np.delete(indices, np.concatenate(([last], np.where(iou > thresh)[0])))

    return keep

### NMS Scratch

In [9]:
import torch

def YOLODetector(feature_maps, anchors, n_classes, input_shape, compute_loss=False):
    """
    Convert YOLOv3 layer feature maps to bounding box parameters.
    
    Reference: (1) https://github.com/qqwweee/keras-yolo3/blob/master/yolo3/model.py
               (2) https://github.com/jiasenlu/YOLOv3.pytorch/blob/master/misc/yolo.py
    
    Parameters
    ----------
    feature_maps: Feature maps learned by the YOLOv3 layer, shape = [1, 3*(5+C), 13, 13]
    anchors: Numpy array of shape = (3, 2). 3 anchors for each scale, and an anchor
        specifies its [width, height]. There are total 9 anchors, 3 for each scale.
    n_classes: int, number of classes
    input_shape: Pytorch tensor, that specifies (height, width). NOTE: height and width 
        are multiples of 32
    compute_loss: bool, if True then return outputs to calculate loss, else return
        predictions
    
    Return
    ------
    If compute loss is true then:
        grid (cell offsets), size: [1, 13, 13, 1, 2], where [..., 2:] is x,y center of cells
        feature_maps: Feature maps (raw predictions) learned by the YOLOv3 layer, size: [1, 13, 13, 3, 5+C]
        box_xy: Center (x, y) of bounding box, size: [1, 13, 13, 3, 2]
        box_wh: width, height of bounding box, size: [1, 13, 13, 3, 2]
    else:
        box_xy: Center (x, y) of bounding box, size: [1, 13, 13, 3, 2]
        box_wh: width, height of bounding box, size: [1, 13, 13, 3, 2]
        box_confidence: Confidence score, size: [1, 13, 13, 3, 1]
        box_class_probs: Class probabilities, size: [1, 13, 13, 3, C]
    """
    # NOTE: Comments are based on feature_maps of size [N, 3*(5+C), 13, 13] 
    if not compute_loss:
        feature_maps = feature_maps.cpu()
        input_shape = input_shape.cpu()
        
    # Number of anchors for each scale. It should be 3 anchors in each scale
    num_anchors = len(anchors)  # 3
    
    # Convert NumPy array to Torch tensor and reshape to include dimensions for (num_images, height, 
    # width, scales, 5+C), size: [3, 2] -> [1, 1, 1, 3, 2]
    anchors_tensor = torch.from_numpy(anchors).view(1, 1, 1, num_anchors, 2).type_as(feature_maps)
    
    # Compute grid shape
    grid_shape = feature_maps.shape[2:4]  # height x width
    
    # Create a grid or cell offsets
    grid_y = torch.arange(0, grid_shape[0])  # size: [13]
    grid_x = torch.arange(0, grid_shape[1])  # size: [13]

    grid_y = grid_y.view(-1, 1, 1, 1)  # size: [13] -> [13, 1, 1, 1]
    grid_x = grid_y.view(1, -1, 1, 1)  # size: [13] -> [1, 13, 1, 1]
    
    grid_y = grid_y.expand(grid_shape[0], grid_shape[0], 1, 1)  # size: [13, 1, 1, 1] -> [13, 13, 1, 1]
    grid_x = grid_x.expand(grid_shape[1], grid_shape[1], 1, 1)  # size: [1, 13, 1, 1] -> [13, 13, 1, 1]
    
    # Grid (x, y), where (x, y) is center of cell. Check `grid[0:2, ...]` output
    #  (0,0) (1,0) ... (12,0)
    #  (0,1) (1,1) ... ...
    #  ...         ... ...
    #  (0,12) ...  ... (12,12)
    grid = torch.cat([grid_x, grid_y], dim=3)  # size: [13, 13, 1, 2]
    
    # Insert one dimension for batch size
    grid = grid.unsqueeze(0).type_as(feature_maps)  # size: [13, 13, 1, 2] -> [1, 13, 13, 1, 2]
    
    # Reshape feature maps size: [1, 3*(5+C), 13, 13] -> [1, 13, 13, 3, 5+C]
    feature_maps = feature_maps.view(-1, num_anchors, 5 + n_classes, grid_shape[0], grid_shape[1])  # size: [1, 3*(5+C), 13, 13] -> [1, 3, 5+C, 13, 13]
    feature_maps = feature_maps.permute(0, 3, 4, 1, 2).contiguous()  # size: # [1, 3, 5+C, 13, 13] -> [1, 13, 13, 3, 5+C]
    
    # Compute: bx = sigmoid(tx) + cx and by = sigmoid(ty) + cy, output size: [1, 13, 13, 3, 2]
    box_xy = torch.sigmoid(feature_maps[..., :2]) + grid  # feature_maps[...,:2] -> xy
    
    # Compute: bw = pw * exp(tw) and bh = ph * exp(th), output size: [1, 13, 13, 3, 2]
    box_wh = anchors_tensor * torch.exp(feature_maps[..., 2:4])  # feature_maps[...,2:4] -> wh
    
    # Adjust predictions to each spatial grid point and anchor size
    # box_xy some values are > 1 so [sigmoid(tx) + cx]/13 and [sigmoid(ty) + cy]/13
    # makes box_xy values to be in range [0, 1]
    box_xy = box_xy / torch.tensor(grid_shape).view(1, 1, 1, 1, 2).type_as(feature_maps)
    
    # box_wh values needs to be scaled by input_shape
    box_wh = box_wh / input_shape.view(1, 1, 1, 1, 2)
    
    # Box confidence score, output size: [1, 13, 13, 3, 1]
    box_confidence = torch.sigmoid(feature_maps[..., 4:5]) # feature_maps[..., 4:5] -> confidence scores
    
    # Box class probabilities, output size: [1, 13, 13, 3, C]
    box_class_probs = torch.sigmoid(feature_maps[..., 5:]) # feature_maps[..., 5:] -> class scores
    
    if compute_loss:
        return grid, feature_maps, box_xy, box_wh
    return box_xy, box_wh, box_confidence, box_class_probs


def yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape):
    """
    Convert YOLO bounding box predictions to bounding box coordinates (x_min,
    y_min, x_max, y_max)
    
    Parameters
    ----------
    box_xy: PyTorch tensor, box_xy output from YOLODetector, size: [1, 13, 13, 3, 2]
    box_wh: PyTorch tensor, box_wh output from YOLODetector, size: [1, 13, 13, 3, 2]
    input_shape: ? e.g. 416x416
    image_shape: ? e.g. 640x480
    """
    # [x, y] -> [y, x]
    box_yx = torch.stack((box_xy[..., 1], box_xy[..., 0]), dim=4)
    # [w, h] -> [h, w]
    box_hw = torch.stack((box_wh[..., 1], box_wh[..., 0]), dim=4)
    
    factor = torch.min((input_shape / image_shape))  # min(416./640., 416./480.)
    
    # New shape: round(640. * 416./640., 480. * 416./640.)
    new_shape = torch.round(image_shape * factor)
    
    # Compute offset: [0., (416.-312.)/(2*416.)] i.e. [0, 0.125]
    offset = (input_shape - new_shape) / (2. * input_shape)
    
    # Compute scale: [1., 416./312.] i.e. [1., 1.33]
    scale = input_shape / new_shape
    
    # Convert boxes from center (y,x) and (h, w) to (y_min, x_min) and (y_max, x_max)
    box_yx = (box_yx - offset) * scale  # [(x-0.)*1., (y-0.125)*1.33]
    box_hw = box_hw * scale  # [h*1, w*1.33]
    
    box_mins = box_yx - (box_hw / 2.)  # x_min = (x-0.)*1. - h/2, y_min = ...
    box_maxes = box_yx + (box_hw / 2.)  # x_max = (x-0.)*1. + h/2, y_max = ...
    
    # Stack box coordinates in proper order
    boxes = torch.stack([
        box_mins[..., 0], # y_min
        box_mins[..., 1], # x_min
        box_maxes[..., 0], # y_max
        box_maxes[..., 1], # x_max
    ], dim=4)  # size: [1, 13, 13, 3, 4]
    
    # Scale boxes back to original image shape
    boxes = boxes * torch.cat([image_shape, image_shape]).view(1, 1, 1, 1, 4)
    
    return boxes


def yolo_boxes_and_scores(feature_maps, anchors, n_classes, input_shape, image_shape):
    """
    Process output from YOLODetector
    
    Parameters
    ----------
    feature_maps: Feature maps learned by the YOLOv3 layer, shape = [1, 3*(5+C), 13, 13]
    anchors: Numpy array of shape = (3, 2). 3 anchors for each scale, and an anchor
        specifies its [width, height]. There are total 9 anchors, 3 for each scale.
    n_classes: int, number of classes
    input_shape: Pytorch tensor, that specifies (height, width). NOTE: height and width 
        are multiples of 32
    image_shape: Pytorch tensor?
    
    Return
    ------
    """
    # Get output from YOLODetector
    box_xy, box_wh, box_confidence, box_class_probs = YOLODetector(feature_maps, anchors, n_classes, input_shape)
    
    # Correct the bounding boxes, size: [N, 13, 13, 3, 4] where 4 specifies y_min, x_min, y_max, x_max
    boxes = yolo_correct_boxes(box_xy, box_wh, input_shape, image_shape)
    
    # Resize boxes tensor, size: [N, 13, 13, 3, 4] -> [13 * 13 * num_scales, 4]
    boxes = boxes.view([-1, 4])
    
    # Box scores = Box confidence * Box class probabilities
    box_scores = box_confidence * box_class_probs  # size: [N, 13, 13, 3, 4]
    box_scores = box_scores.view(-1, n_classes)  # size: [13 * 13 * num_scales, n_classes]
    
    return boxes.view(feature_maps.size(0), -1, 4), box_scores.view(feature_maps.size(0), -1, n_classes)

In [12]:
# Features and other inputs
out13 = torch.randn([1, 27, 13, 13])

n_classes = 4
input_shape = [416., 416.]
anchors = np.array([[10, 13], [16, 30], [33, 23], 
                    [30, 61], [62, 45], [59, 119], 
                    [116, 90], [156, 198], [373, 326]])

anchor_mask = [[6, 7, 8], [3, 4, 5], [0, 1, 2]]
ANCHORS = anchors[anchor_mask[0]]

boxes, box_scores = yolo_boxes_and_scores(out13, ANCHORS, n_classes, torch.tensor([416., 416.]), torch.tensor([640., 480.]))

In [14]:
BOXES = []
BOX_SCORES = []

BOXES.append(boxes)
BOX_SCORES.append(box_scores)

boxes = torch.cat(BOXES, dim=1)
box_scores = torch.cat(BOX_SCORES, dim=1)

print(boxes.shape)
print(box_scores.shape)

torch.Size([1, 507, 4])
torch.Size([1, 507, 4])


In [52]:
i = 0  # Each image
thresh = 0.3
mask = box_scores[i] >= thresh
print('mask size: ', mask.shape)
print()

for c in range(n_classes):
    print('keep boxes for class "{}: {}"'.format(c, boxes[i][mask[:, c]].shape))
    print('junk boxes for class "{}: {}"'.format(c, boxes[i][~mask[:, c]].shape))
    print()
    
    # For each class keep boxes and scores that have score >= threshold
    class_boxes = boxes[i][mask[:, c]]  # [?, 4]
    class_box_scores = box_scores[i][:, c][mask[:, c]]
    
    _, order = torch.sort(class_box_scores, dim=0, descending=True)
    
    class_detections = torch.cat((class_boxes, class_box_scores.view(-1, 1)), dim=1)
    class_detections = class_detections[order]

mask size:  torch.Size([507, 4])

keep boxes for class "0: torch.Size([177, 4])"
junk boxes for class "0: torch.Size([330, 4])"

keep boxes for class "1: torch.Size([175, 4])"
junk boxes for class "1: torch.Size([332, 4])"

keep boxes for class "2: torch.Size([158, 4])"
junk boxes for class "2: torch.Size([349, 4])"

keep boxes for class "3: torch.Size([166, 4])"
junk boxes for class "3: torch.Size([341, 4])"



In [42]:
box_scores[0].shape

torch.Size([507, 4])

In [44]:
box_scores[0][:, 0].shape

torch.Size([507])

In [48]:
box_scores[0][:, 0][mask[:, 0]].shape

torch.Size([177])

In [70]:
boxes = class_detections.numpy()
print(boxes.shape)

(166, 5)


In [75]:
boxes[:, 4]

array([0.77761626, 0.6930929 , 0.6824322 , 0.66286105, 0.6550815 ,
       0.6467649 , 0.64640534, 0.6378872 , 0.63715166, 0.6080906 ,
       0.5972927 , 0.59722424, 0.5968263 , 0.5923946 , 0.59104806,
       0.5841972 , 0.5775911 , 0.57643837, 0.5720977 , 0.5615593 ,
       0.5528987 , 0.5511499 , 0.5497742 , 0.5459416 , 0.5436691 ,
       0.5417604 , 0.54038274, 0.54027116, 0.5369562 , 0.53452015,
       0.5328853 , 0.5290111 , 0.5249108 , 0.5228807 , 0.52003896,
       0.51698977, 0.51007205, 0.5043706 , 0.5042956 , 0.50385225,
       0.5001142 , 0.49920788, 0.49545673, 0.49102738, 0.48858812,
       0.4884097 , 0.4798831 , 0.47283164, 0.46537045, 0.464531  ,
       0.4642155 , 0.45790142, 0.4562222 , 0.45486662, 0.4527519 ,
       0.4501553 , 0.4490366 , 0.44821596, 0.44147503, 0.44135502,
       0.4393673 , 0.43831062, 0.43663472, 0.4310224 , 0.42985553,
       0.4292596 , 0.42617965, 0.42557174, 0.42508066, 0.42376757,
       0.4197288 , 0.41908574, 0.41837496, 0.4178247 , 0.41746

In [76]:
order = boxes[:, 4].argsort()
order

array([165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153,
       152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140,
       139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127,
       126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114,
       113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101,
       100,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,  88,
        87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,  75,
        74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,  62,
        61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,  49,
        48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,  36,
        35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,
         9,   8,   7,   6,   5,   4,   3,   2,   1,   0])

In [77]:
last = len(order) - 1
print(last)

i = order[last]
print(i)

165
0


In [80]:
order[:last]

array([165, 164, 163, 162, 161, 160, 159, 158, 157, 156, 155, 154, 153,
       152, 151, 150, 149, 148, 147, 146, 145, 144, 143, 142, 141, 140,
       139, 138, 137, 136, 135, 134, 133, 132, 131, 130, 129, 128, 127,
       126, 125, 124, 123, 122, 121, 120, 119, 118, 117, 116, 115, 114,
       113, 112, 111, 110, 109, 108, 107, 106, 105, 104, 103, 102, 101,
       100,  99,  98,  97,  96,  95,  94,  93,  92,  91,  90,  89,  88,
        87,  86,  85,  84,  83,  82,  81,  80,  79,  78,  77,  76,  75,
        74,  73,  72,  71,  70,  69,  68,  67,  66,  65,  64,  63,  62,
        61,  60,  59,  58,  57,  56,  55,  54,  53,  52,  51,  50,  49,
        48,  47,  46,  45,  44,  43,  42,  41,  40,  39,  38,  37,  36,
        35,  34,  33,  32,  31,  30,  29,  28,  27,  26,  25,  24,  23,
        22,  21,  20,  19,  18,  17,  16,  15,  14,  13,  12,  11,  10,
         9,   8,   7,   6,   5,   4,   3,   2,   1])

In [109]:
non_maximum_suppression(boxes, thresh=0.3)

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 8,
 9,
 10,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 25,
 26,
 28,
 29,
 30,
 31,
 33,
 34,
 35,
 36,
 38,
 41,
 42,
 43,
 44,
 45,
 49,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 69,
 80,
 84,
 85,
 86,
 89,
 94,
 95,
 97,
 105,
 106,
 107,
 109,
 110,
 115,
 116,
 119,
 120,
 122,
 126,
 127,
 133,
 137,
 141,
 144,
 145,
 146,
 147,
 148,
 149,
 154,
 159,
 163,
 164,
 165]